In [1]:
import numpy as np
import pandas as pd

In [3]:
iterables = [['bar', 'baz'], ['one', 'two'], ['dog', 'cat']]

df = pd.DataFrame(np.random.randn(2, 8), columns=pd.MultiIndex.from_product(iterables, names=['first', 'second', 'third']))

#df.sum(axis=1, level=['first', 'third'], skipna=False)

In [12]:
df.mean(axis=1, level=['first', 'third'], skipna=False)

ValueError: No axis named 1 for object type Series